In [1]:
import os
%pwd
os.chdir("../")
%pwd

'/Users/niralpatel/Desktop/Projects/speaking_silence'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path  # Root directory of the project
    base_model_path: Path  # Path to save the base model
    updated_base_model_path: Path  # Path to save the updated base model

    input_shape: tuple  # Input shape of the model (e.g., (height, width, channels))
    num_classes: int  # Number of gesture classes
    learning_rate: float  # Learning rate for model training
    include_top: bool  # Whether to include the top layers (True for fine-tuning)


In [10]:
from Speaking_Silence.constants import *
from Speaking_Silence.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            input_shape=self.params.INPUT_SHAPE,
            learning_rate=self.params.LEARNING_RATE,
            include_top=self.params.INCLUDE_TOP,
            num_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from pathlib import Path

In [12]:


class PrepareBaseModel:
    def __init__(self, config):
        self.config = config

    def build_base_model(self):
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=self.config.input_shape),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(self.config.num_classes, activation='softmax')
        ])
        return model

    def save_model(self, model, path):
        model.save(path)

    def prepare_and_save_base_model(self):
        base_model = self.build_base_model()
        self.save_model(base_model, self.config.base_model_path)


In [14]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.prepare_and_save_base_model()
except Exception as e:
    raise e

[2024-03-22 23:02:26,949: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-22 23:02:26,953: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-22 23:02:26,954: INFO: common: created directory at: artifacts]
[2024-03-22 23:02:26,955: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-03-22 23:02:27,079: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


/Users/niralpatel/Desktop/Projects/speaking_silence/env/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
